In [1]:
# ARC Prize 2025 – ULTIMATE RECURSIVE MULTI-MODAL ARCHITECTURE

# Cell 1: Setup & Paths
import os, json, itertools, math, time, random
from collections import Counter, defaultdict, deque
from copy import deepcopy
import numpy as np

# Determinism
random.seed(42)
np.random.seed(42)

# Kaggle paths
TRAIN_JSON = "/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json"
TEST_JSON  = "/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json"
OUTPUT_DIR = "/kaggle/working"
SUBMISSION_PATH = os.path.join(OUTPUT_DIR, "submission.json")

# Runtime knobs
MAX_TIME_PER_TASK = 3.0    # seconds per task
USE_COMPOSITIONS = True    # enable depth-2 compositions
PRINT_PROGRESS_EVERY = 50

# Operation categorization for enhanced diversity ranking (Cell 5)
OP_CATEGORIES = {
    'identity': ['op_identity'],
    'rotation': ['op_rotate90', 'op_rotate180', 'op_rotate270'],
    'reflection': ['op_mirror_h', 'op_mirror_v', 'op_transpose'],
    'cropping': ['op_crop_center_pad', 'op_largest_cc_center', 'op_trim_to_non_background_grid'],
    'recolor': ['op_recolor_learned', 'op_invert_colors', 'op_normalize_color_to_1', 'op_recolor_by_ratio'],
    'structural': ['op_majority_fill', 'op_tile_smallest_patch', 'op_fill_holes', 'op_complete_by_symmetry'],
    'cc_manip': ['op_keep_largest_cc_only', 'op_grow_largest_cc', 'op_shrink_largest_cc', 'op_replace_cc_with_pixel', 'op_process_blocks'],
    'io_abstraction': ['op_io_global_color_map'],
    'object_manip': ['op_recursive_cc_transformation'],  # NEW SOLVER C
}
def get_op_category(lbl):
    name = lbl.split(':')[-1].split('+')[0]
    for cat, ops in OP_CATEGORIES.items():
        if name in ops: return cat
    return 'other'

# Cell 2: Robust ARC Task Loader
def _is_task(obj):
    return isinstance(obj, dict) and ("train" in obj) and ("test" in obj)

def _looks_like_task_dict(d):
    if not isinstance(d, dict) or not d:
        return False
    sample_vals = list(d.values())[:5]
    return all(_is_task(v) for v in sample_vals if isinstance(v, dict))

def _unwrap_wrappers(obj):
    if isinstance(obj, dict):
        for key in ["tasks", "task_list", "training", "train_tasks", "data", "items"]:
            if key in obj and isinstance(obj[key], list):
                return obj[key]
    return obj

def load_arc_tasks_anyshape(path):
    with open(path, "r") as f:
        raw = f.read().strip()

    if not (raw.startswith("{") or raw.startswith("[")):
        out = []
        for line in raw.splitlines():
            s = line.strip()
            if not s:
                continue
            o = json.loads(s)
            if _is_task(o):
                out.append(o)
            else:
                o = _unwrap_wrappers(o)
                if isinstance(o, list) and o and _is_task(o[0]):
                    out.extend(o)
                elif isinstance(o, dict) and _looks_like_task_dict(o):
                    out.extend(list(o.values()))
        if not out:
            raise ValueError("Could not parse JSONL into ARC tasks.")
        return out

    obj = json.loads(raw)
    if _is_task(obj):
        return [obj]
    obj = _unwrap_wrappers(obj)
    if isinstance(obj, list):
        if len(obj) == 1 and not _is_task(obj[0]):
            inner = _unwrap_wrappers(obj[0])
            if isinstance(inner, list) and inner and _is_task(inner[0]):
                return inner
        if obj and _is_task(obj[0]):
            return obj
        merged = []
        for it in obj:
            if _is_task(it):
                merged.append(it)
            elif isinstance(it, dict) and _looks_like_task_dict(it):
                merged.extend(list(it.values()))
        if merged:
            return merged

    if isinstance(obj, dict) and _looks_like_task_dict(obj):
        return list(obj.values())

    raise ValueError(f"Unrecognized ARC format for {path}")

# Cell 3: Core grid utilities
def to_np(grid): return np.array(grid, dtype=int)
def to_list(arr): return arr.tolist()
def bg_color(arr):
    vals, cnts = np.unique(arr, return_counts=True)
    return int(vals[np.argmax(cnts)]) if len(vals)>0 else 0
def hamming(a,b):
    if a.shape != b.shape: return 10**9
    return int(np.sum(a!=b))
def rotate(arr,k=1): return np.rot90(arr,k=k)
def mirror_h(arr): return arr[:, ::-1]
def mirror_v(arr): return arr[::-1, :]
def transpose(arr): return arr.T
def crop_to_bbox(arr, bg=None):
    if bg is None: bg = bg_color(arr)
    ys, xs = np.where(arr != bg)
    if len(ys)==0: return arr.copy()
    y0,y1 = ys.min(), ys.max()
    x0,x1 = xs.min(), xs.max()
    return arr[y0:y1+1, x0:x1+1]
def bbox_of_pixels(pixels):
    if not pixels: return 0,0,0,0
    ys=[p[0] for p in pixels]; xs=[p[1] for p in pixels]
    return min(ys), min(xs), max(ys), max(xs)

def pad_to(arr, H, W, bg=None, align='center'):
    if bg is None: bg = bg_color(arr)
    h, w = arr.shape
    out = np.full((H, W), bg, dtype=int)
    if align == 'center':
        dy = (H - h) // 2; dx = (W - w) // 2
    else:  # top-left
        dy = 0; dx = 0
    y0 = max(0, dy); x0 = max(0, dx)
    sy = max(0, -dy); sx = max(0, -dx)
    copy_h = min(H - y0, h - sy)
    copy_w = min(W - x0, w - sx)
    if copy_h > 0 and copy_w > 0:
        out[y0:y0+copy_h, x0:x0+copy_w] = arr[sy:sy+copy_h, sx:sx+copy_w]
    return out

def connected_components(arr, bg=None):
    if bg is None: bg = bg_color(arr)
    H,W = arr.shape
    seen = np.zeros((H,W), dtype=bool)
    comps=[]
    for y in range(H):
        for x in range(W):
            if seen[y,x] or arr[y,x]==bg: continue
            color = arr[y,x]
            stack=deque([(y,x)]); seen[y,x]=True; pix=[]
            while stack:
                cy,cx = stack.popleft()
                pix.append((cy,cx))
                for dy,dx in [(1,0),(-1,0),(0,1),(0,-1)]:
                    ny,nx = cy+dy, cx+dx
                    if 0<=ny<H and 0<=nx<W and not seen[ny,nx] and arr[ny,nx]==color:
                        seen[ny,nx]=True; stack.append((ny,nx))
            comps.append((color,pix))
    return comps

def majority_color(arr):
    vals, cnts = np.unique(arr, return_counts=True)
    return int(vals[np.argmax(cnts)]) if len(vals)>0 else 0

def resize_integer_scale(obj_arr, target_h, target_w, bg=None):
    if bg is None: bg = bg_color(obj_arr)
    h,w = obj_arr.shape
    if h==0 or w==0:
        return np.full((target_h,target_w), bg, dtype=int)
    sy = max(1, round(target_h / h))
    sx = max(1, round(target_w / w))
    scaled = np.repeat(np.repeat(obj_arr, sy, axis=0), sx, axis=1)
    sh,sw = scaled.shape
    if sh>=target_h and sw>=target_w:
        y0=(sh-target_h)//2; x0=(sw-target_w)//2
        scaled = scaled[y0:y0+target_h, x0:x0+sw]
    else:
        scaled = pad_to(scaled, target_h, target_w, bg=bg, align='center')
    return scaled

def smallest_tile(arr):
    H,W = arr.shape
    for th in range(1,H+1):
        if H%th: continue
        for tw in range(1,W+1):
            if W%tw: continue
            tile = arr[0:th,0:tw]
            if np.array_equal(np.tile(tile,(H//th, W//tw)), arr):
                return tile
    return None

def dilate(arr, bg=None, k=1):
    if bg is None: bg = bg_color(arr)
    H, W = arr.shape
    out = arr.copy()
    for _ in range(k):
        next_out = out.copy()
        ys, xs = np.where(out != bg)
        for y, x in zip(ys, xs):
            color = out[y, x]
            for dy, dx in [(0,1), (0,-1), (1,0), (-1,0)]:
                ny, nx = y + dy, x + dx
                if 0 <= ny < H and 0 <= nx < W and out[ny, nx] == bg:
                    next_out[ny, nx] = color
        out = next_out
    return out

def erode(arr, bg=None, k=1):
    if bg is None: bg = bg_color(arr)
    H, W = arr.shape
    out = arr.copy()
    for _ in range(k):
        next_out = out.copy()
        for y in range(H):
            for x in range(W):
                if out[y, x] != bg:
                    is_edge = False
                    for dy, dx in [(0,1), (0,-1), (1,0), (-1,0)]:
                        ny, nx = y + dy, x + dx
                        if not (0 <= ny < H and 0 <= nx < W) or out[ny, nx] == bg:
                            is_edge = True
                            break
                    if is_edge:
                        next_out[y, x] = bg
        out = next_out
    return out

def decompose_into_blocks(arr, target_h, target_w):
    H, W = arr.shape
    blocks = []

    if H % target_h == 0 and W % target_w == 0:
        for r in range(0, H, target_h):
            for c in range(0, W, target_w):
                blocks.append(arr[r:r+target_h, c:c+target_w])
        return blocks

    if H > 1 and W > 1:
        for th in range(1, min(H, 5)):
            if H % th == 0:
                for tw in range(1, min(W, 5)):
                    if W % tw == 0:
                        blocks = []
                        for r in range(0, H, th):
                            for c in range(0, W, tw):
                                blocks.append(arr[r:r+th, c:c+tw])
                        return blocks
    return [arr]

# Cell 4: Heuristic operations (ULTIMATE FINAL UPGRADE - Recursive CC Solver Added)
def learn_color_map(train_pairs):
    mapping={}
    for xin,xout in train_pairs:
        cin=Counter(xin.flatten()); cout=Counter(xout.flatten())
        if not cin or not cout: continue
        bg_in=max(cin,key=cin.get); bg_out=max(cout,key=cout.get)
        in_sorted =[c for c,_ in cin.most_common() if c!=bg_in]
        out_sorted=[c for c,_ in cout.most_common() if c!=bg_out]
        for i,c in enumerate(in_sorted):
            if i<len(out_sorted): mapping[c]=out_sorted[i]
        mapping[bg_in]=bg_out
    return mapping

def apply_color_map(arr,cmap):
    out=arr.copy()
    for ci,co in cmap.items(): out[arr==ci]=co
    return out

# --- CORE BASE OPS ---
def op_identity(arr,H,W,train_pairs=None): return pad_to(arr,H,W,bg=bg_color(arr))
def op_rotate90(arr,H,W,train_pairs=None): return pad_to(rotate(arr,1),H,W,bg=bg_color(arr))
def op_rotate180(arr,H,W,train_pairs=None): return pad_to(rotate(arr,2),H,W,bg=bg_color(arr))
def op_rotate270(arr,H,W,train_pairs=None): return pad_to(rotate(arr,3),H,W,bg=bg_color(arr))
def op_mirror_h(arr,H,W,train_pairs=None): return pad_to(mirror_h(arr),H,W,bg=bg_color(arr))
def op_mirror_v(arr,H,W,train_pairs=None): return pad_to(mirror_v(arr),H,W,bg=bg_color(arr))
def op_transpose(arr,H,W,train_pairs=None): return pad_to(transpose(arr),H,W,bg=bg_color(arr))
def op_crop_center_pad(arr,H,W,train_pairs=None):
    return pad_to(crop_to_bbox(arr,bg=bg_color(arr)), H, W, bg=bg_color(arr), align='center')
def op_recolor_learned(arr,H,W,train_pairs=None):
    cmap = learn_color_map(train_pairs or [])
    recol = apply_color_map(arr,cmap)
    return pad_to(recol,H,W,bg=bg_color(recol))
def op_keep_largest_cc_only(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg)
    comps = connected_components(a, bg=bg)
    if not comps: return a
    _, pix = max(comps, key=lambda cp: len(cp[1]))
    out = np.full_like(a, bg)
    for (y,x) in pix: out[y,x] = a[y,x]
    return out
def op_tile_smallest_patch(arr,H,W,train_pairs=None):
    tile=smallest_tile(arr)
    if tile is None: return pad_to(arr,H,W,bg=bg_color(arr))
    th,tw=tile.shape
    tiled=np.tile(tile,(max(1,math.ceil(H/th)), max(1,math.ceil(W/tw))))
    return tiled[:H,:W]

# --- ADVANCED STRUCTURAL/MANIPULATION OPS ---
def op_grow_largest_cc(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg)
    comps = connected_components(a, bg=bg)
    if not comps: return a
    _, pix = max(comps, key=lambda cp: len(cp[1]))
    mask = np.full_like(a, bg)
    for y, x in pix: mask[y, x] = a[y, x]
    return pad_to(dilate(mask, bg=bg, k=1), H, W, bg=bg)
def op_shrink_largest_cc(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg)
    comps = connected_components(a, bg=bg)
    if not comps: return a
    _, pix = max(comps, key=lambda cp: len(cp[1]))
    mask = np.full_like(a, bg)
    for y, x in pix: mask[y, x] = a[y, x]
    return pad_to(erode(mask, bg=bg, k=1), H, W, bg=bg)
def op_invert_colors(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg); out = a.copy()
    vals = [c for c in np.unique(a) if c != bg]
    if len(vals) >= 2:
        counts = Counter(a.flatten())
        sorted_vals = sorted(vals, key=lambda c: counts.get(c, 0), reverse=True)
        c1, c2 = sorted_vals[0], sorted_vals[1]
        out[a == c1] = c2
        out[a == c2] = c1
    return out
def op_normalize_color_to_1(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg); out = a.copy()
    unique_non_bg = [c for c in np.unique(a) if c != bg]
    if not unique_non_bg: return out
    counts = Counter(a.flatten())
    sorted_colors = sorted(unique_non_bg, key=lambda c: counts.get(c, 0), reverse=True)
    new_color_map = {bg: bg}; target_color = 1
    for old_color in sorted_colors:
        if old_color == bg: continue
        while target_color == bg: target_color = (target_color % 9) + 1
        new_color_map[old_color] = target_color
        target_color = (target_color % 9) + 1
    for old_c, new_c in new_color_map.items():
        out[a == old_c] = new_c
    return out
def op_replace_cc_with_pixel(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg);
    comps = connected_components(a, bg=bg);
    out = np.full_like(a, bg);
    for color, pix in comps:
        if not pix: continue
        y0, x0, y1, x1 = bbox_of_pixels(pix)
        cy = (y0 + y1) // 2
        cx = (x0 + x1) // 2
        out[cy, cx] = color
    return out
def op_complete_by_symmetry(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg); Hh, Ww = a.shape; out = a.copy()
    L = a[:, :Ww//2]; R_candidate = a[:, Ww - (Ww//2):]
    if Ww % 2 != 0: C = a[:, Ww//2]; R_candidate = a[:, Ww//2 + 1:]
    if np.count_nonzero(L != bg) > 0.5 * Ww * Hh and np.count_nonzero(R_candidate != bg) < 0.1 * Ww * Hh:
        mirror = mirror_h(L)
        if Ww % 2 != 0: out[:, Ww//2 + 1:] = mirror
        else: out[:, Ww//2:] = mirror
        return out
    T = a[:Hh//2, :]; B_candidate = a[Hh - (Hh//2):, :]
    if np.count_nonzero(T != bg) > 0.5 * Ww * Hh and np.count_nonzero(B_candidate != bg) < 0.1 * Ww * Hh:
        mirror = mirror_v(T)
        if Hh % 2 != 0: out[Hh//2 + 1:, :] = mirror
        else: out[Hh//2:, :] = mirror
        return out
    return a
def op_recolor_by_ratio(arr, H, W, train_pairs=None):
    if not train_pairs: return pad_to(arr, H, W, bg=bg_color(arr))
    input_sizes = [x.size for x, _ in train_pairs]
    output_sizes = [y.size for _, y in train_pairs]
    if not input_sizes or not output_sizes: return pad_to(arr, H, W, bg=bg_color(arr))
    avg_ratio = np.mean(output_sizes) / np.mean(input_sizes)
    if avg_ratio > 4.0: target_color = 3
    elif avg_ratio > 1.5: target_color = 2
    elif avg_ratio < 0.5: target_color = 1
    else: return op_recolor_learned(arr, H, W, train_pairs)
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg); out = a.copy()
    out[out != bg] = target_color
    return out
def op_process_blocks(arr, H, W, train_pairs=None):
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg)
    train_H = [x.shape[0] for x, _ in train_pairs]
    train_W = [x.shape[1] for x, _ in train_pairs]
    if len(train_H) < 2 or len(set(train_H)) < 2: bh, bw = 3, 3
    else: bh = min(train_H); bw = min(train_W)
    blocks = decompose_into_blocks(a, bh, bw)
    if len(blocks) < 2: return a
    major_color = majority_color(blocks[0])
    out = np.full_like(a, major_color)
    return out
def op_trim_to_non_background_grid(arr, H, W, train_pairs=None):
    """Crops the grid to only the non-background components, then pads back."""
    bg = bg_color(arr);
    cropped = crop_to_bbox(arr, bg=bg)
    # Apply a learned simple transform to the cropped part before padding back
    if train_pairs and len(train_pairs) > 0:
        if cropped.shape == train_pairs[0][0].shape:
            # Try to rotate the cropped part if the IO has a rotation pattern
            if all(np.array_equal(rotate(x, 1), y) for x, y in train_pairs[:2]):
                cropped = rotate(cropped, 1)
    return pad_to(cropped, H, W, bg=bg)

# --- NEW SOLVER C: Recursive Object-Based Transformation ---
def op_recursive_cc_transformation(arr, H, W, train_pairs):
    """Transforms individual connected components (objects) and places them back."""
    bg = bg_color(arr); a = pad_to(arr, H, W, bg=bg)
    comps = connected_components(a, bg=bg)
    out = np.full((H, W), bg, dtype=int)

    # Simple rule learning: Find the transformation applied to the LARGEST object
    if not comps or not train_pairs: return a

    # 1. Identify the rule from the first training pair
    x_train, y_train = train_pairs[0]
    x_comps = connected_components(x_train, bg=bg_color(x_train))
    y_comps = connected_components(y_train, bg=bg_color(y_train))

    if not x_comps or not y_comps: return a

    # Find the largest component in the input
    _, x_pix = max(x_comps, key=lambda cp: len(cp[1]))
    x_y0, x_x0, x_y1, x_x1 = bbox_of_pixels(x_pix)
    x_obj = crop_to_bbox(x_train[x_y0:x_y1+1, x_x0:x_x1+1], bg=bg_color(x_train))

    # Trying to find the corresponding largest component in the output (by bounding box overlap/color)
    best_match_y = None
    max_overlap = -1
    for y_color, y_pix in y_comps:
        y_y0, y_x0, y_y1, y_x1 = bbox_of_pixels(y_pix)
        if len(y_pix) > max_overlap:
            max_overlap = len(y_pix)
            best_match_y = y_train[y_y0:y_y1+1, y_x0:y_x1+1]

    if best_match_y is None: return a

    y_obj = crop_to_bbox(best_match_y, bg=bg_color(y_train))

    # 2. Infer the transformation (simple check: rotate/mirror)
    rule = op_identity
    if x_obj.shape == y_obj.shape:
        if hamming(rotate(x_obj, 1), y_obj) == 0: rule = op_rotate90
        elif hamming(mirror_h(x_obj), y_obj) == 0: rule = op_mirror_h
        elif hamming(mirror_v(x_obj), y_obj) == 0: rule = op_mirror_v

    # 3. Apply the rule to all components in the test input
    for color, pix in comps:
        y0, x0, y1, x1 = bbox_of_pixels(pix)
        component_grid = arr[y0:y1+1, x0:x1+1]
        transformed_grid = rule(component_grid, component_grid.shape[0], component_grid.shape[1], None)
        h, w = transformed_grid.shape
        out[y0:y0+h, x0:x0+w] = transformed_grid

    return pad_to(out, H, W, bg=bg)

# Aggregate ALL operations
BASIC_OPS = [
    op_identity, op_rotate90, op_rotate180, op_rotate270,
    op_mirror_h, op_mirror_v, op_transpose,
    op_crop_center_pad, op_recolor_learned,
    op_keep_largest_cc_only, op_tile_smallest_patch,
    # Advanced Ops
    op_grow_largest_cc, op_shrink_largest_cc,
    op_invert_colors, op_normalize_color_to_1,
    op_replace_cc_with_pixel,
    # Ultimate Ops
    op_complete_by_symmetry,
    op_recolor_by_ratio,
    op_process_blocks,
    op_trim_to_non_background_grid,
]

# Cell 5: candidate generation, survivor selection, signature, ranking

# --- SOLVER B: IO-to-IO Abstraction (non-CC solver) ---
def op_io_global_color_map(arr, H, W, train_pairs):
    if not train_pairs: return pad_to(arr, H, W, bg=bg_color(arr))
    all_mappings = defaultdict(lambda: Counter())
    for x_in, y_out in train_pairs:
        bg_in = bg_color(x_in); bg_out = bg_color(y_out)
        x_fg_colors = [c for c in np.unique(x_in) if c != bg_in]
        y_fg_colors = [c for c in np.unique(y_out) if c != bg_out]
        in_counts = Counter(x_in.flatten())
        out_counts = Counter(y_out.flatten())
        in_ranked = sorted(x_fg_colors, key=lambda c: in_counts[c], reverse=True)
        out_ranked = sorted(y_fg_colors, key=lambda c: out_counts[c], reverse=True)
        for i, c_in in enumerate(in_ranked):
            if i < len(out_ranked): all_mappings[c_in][out_ranked[i]] += 1
        all_mappings[bg_in][bg_out] += 1
    final_map = {}
    for c_in, c_out_counter in all_mappings.items():
        if c_out_counter: final_map[c_in] = c_out_counter.most_common(1)[0][0]
    recol = apply_color_map(arr, final_map)
    return pad_to(recol, H, W, bg=bg_color(recol))

# --- GENERATE CANDIDATES (Now includes Solver C) ---
def generate_candidates(x_in, H, W, train_pairs, use_compositions=True):
    cands=[]

    # Solver A: Heuristic Compositions
    for f in BASIC_OPS:
        try:
            y=f(x_in,H,W,train_pairs)
            if y.shape==(H,W): cands.append((f"op:{f.__name__}", y))
        except Exception:
            pass
    if use_compositions:
        for f,g in itertools.product(BASIC_OPS, BASIC_OPS):
            try:
                y1=g(x_in,H,W,train_pairs)
                if y1.shape!=(H,W): continue
                y2=f(y1,H,W,train_pairs)
                if y2.shape==(H,W): cands.append((f"op2:{f.__name__}+{g.__name__}", y2))
            except Exception:
                pass

    # Solver B: IO-to-IO Abstraction
    try:
        y_io = op_io_global_color_map(x_in, H, W, train_pairs)
        if y_io.shape==(H,W): cands.append(("io_abstraction:op_io_global_color_map", y_io))
    except Exception:
        pass

    # Solver C: Recursive Object Transformation
    try:
        y_cc = op_recursive_cc_transformation(x_in, H, W, train_pairs)
        if y_cc.shape==(H,W): cands.append(("object_manip:op_recursive_cc_transformation", y_cc))
    except Exception:
        pass

    # Deduplicate candidates
    uniq={}
    for lbl,arr in cands:
        key=arr.tobytes()
        if key not in uniq: uniq[key]=(lbl,arr)
    return list(uniq.values())

# --- UTILITIES ---
def infer_target_shape(train_pairs):
    Hs=[y.shape[0] for _,y in train_pairs]
    Ws=[y.shape[1] for _,y in train_pairs]
    H=Counter(Hs).most_common(1)[0][0]
    W=Counter(Ws).most_common(1)[0][0]
    return H,W

# NEW: Input-aware target shape inference (per test input)
def infer_target_shape_for_input(train_pairs, x_test):
    """Infer the correct (H,W) for this specific test input."""
    Hi, Wi = x_test.shape

    # Collect additive and multiplicative shape relationships from train
    add_rules = []
    mul_rules = []
    for x, y in train_pairs:
        hi, wi = x.shape
        ho, wo = y.shape
        add_rules.append((ho - hi, wo - wi))
        # only keep integer scale rules
        sh = ho / max(1, hi)
        sw = wo / max(1, wi)
        if abs(sh - round(sh)) < 1e-9 and abs(sw - round(sw)) < 1e-9:
            mul_rules.append((int(round(sh)), int(round(sw))))

    # 1) If *all* train pairs keep same size, use the test input size
    if all(dh == 0 and dw == 0 for dh, dw in add_rules):
        return Hi, Wi

    # 2) If all additive deltas are identical, apply that delta
    if len(set(add_rules)) == 1:
        dh, dw = add_rules[0]
        return max(1, Hi + dh), max(1, Wi + dw)

    # 3) If all integer scales are identical, apply that scale
    if mul_rules and len(set(mul_rules)) == 1:
        sh, sw = mul_rules[0]
        return max(1, Hi * sh), max(1, Wi * sw)

    # 4) Majority heuristic
    from collections import Counter as _Counter
    add_major = _Counter(add_rules).most_common(1)[0]
    mul_major = _Counter(mul_rules).most_common(1)[0] if mul_rules else None
    if add_major[1] >= max(2, len(train_pairs)//2):
        dh, dw = add_major[0]
        return max(1, Hi + dh), max(1, Wi + dw)
    if mul_major and mul_major[1] >= max(2, len(train_pairs)//2):
        sh, sw = mul_major[0]
        return max(1, Hi * sh), max(1, Wi * sw)

    # 5) Fallback: keep test input size (safest default)
    return Hi, Wi

def grid_features(a):
    bg = bg_color(a)
    colors = len(np.unique(a))
    ccs = len(connected_components(a, bg=bg))
    filled = int(np.count_nonzero(a != bg))
    return {"colors": colors, "ccs": ccs, "filled": filled}
def learn_output_signature(train_pairs):
    feats = [grid_features(y) for _, y in train_pairs]
    if not feats:
        return {"colors": None, "ccs": None, "filled": None, "best_grid": None}
    colors_med = int(np.median([f["colors"] for f in feats]))
    ccs_med    = int(np.median([f["ccs"]    for f in feats]))
    filled_med = int(np.median([f["filled"] for f in feats]))
    output_grids = [y for _, y in train_pairs]
    output_key = Counter(y.tobytes() for y in output_grids).most_common(1)
    best_grid = None
    if output_key:
        best_bytes = output_key[0][0]
        for y in output_grids:
            if y.tobytes() == best_bytes:
                best_grid = y.tolist()
                break
    return {"colors": colors_med, "ccs": ccs_med, "filled": filled_med, "best_grid": best_grid}
def filter_by_signature(cands, sig, tol_colors=1, tol_ccs=1, tol_filled_ratio=0.6):
    if not sig or sig["colors"] is None: return cands
    kept = []
    for lbl, y in cands:
        f = grid_features(y)
        ok = True
        if abs(f["colors"] - sig["colors"]) > tol_colors: ok = False
        if abs(f["ccs"]    - sig["ccs"])    > tol_ccs:    ok = False
        if sig["filled"] > 0:
            if abs(f["filled"] - sig["filled"]) > max(3, int(sig["filled"] * tol_filled_ratio)): ok = False
        if ok: kept.append((lbl, y))
    return kept if kept else cands

# --- SURVIVOR SELECTION (maps ALL three solvers) ---
def select_survivor_labels(train_pairs, H, W, use_compositions, time_limit=2.0):
    t0=time.time()
    x0,y0=train_pairs[0]
    pool0=generate_candidates(x0,H,W,train_pairs,use_compositions=use_compositions)

    program_map = {p.__name__: p for p in BASIC_OPS}
    program_map['op_io_global_color_map'] = op_io_global_color_map
    program_map['op_recursive_cc_transformation'] = op_recursive_cc_transformation  # Map Solver C

    program_scores = defaultdict(lambda: {'matches': 0, 'weighted_score': -1e9})
    labels = [lbl for lbl,_ in pool0]

    for lbl in labels:
        if time.time()-t0 > time_limit: break

        parts = lbl.split(':')[-1].split('+')
        f_name = parts[0]; g_name = parts[1] if len(parts) > 1 else None

        is_solver_b = (f_name == 'op_io_global_color_map')
        is_solver_c = (f_name == 'op_recursive_cc_transformation')

        if is_solver_b: f = op_io_global_color_map; g = None
        elif is_solver_c: f = op_recursive_cc_transformation; g = None
        else: f = program_map.get(f_name); g = program_map.get(g_name) if g_name else None

        if f is None or (g_name and g is None and g_name != "None"): continue

        current_matches = 0; current_score = 0.0

        for x, y_true in train_pairs:
            try:
                if is_solver_b: y_cand = op_io_global_color_map(x, H, W, train_pairs)
                elif is_solver_c: y_cand = op_recursive_cc_transformation(x, H, W, train_pairs)
                else: y_cand = f(g(x,H,W,train_pairs), H, W, train_pairs) if g else f(x,H,W,train_pairs)

                if y_cand.shape != y_true.shape: current_score -= 100.0
                else:
                    dist = hamming(y_cand, y_true)
                    if dist == 0:
                        current_score += 1000.0
                        current_matches += 1
                    else: current_score -= dist * 0.01
            except Exception: current_score -= 500.0

        program_scores[lbl]['matches'] = current_matches
        program_scores[lbl]['weighted_score'] = current_score

    max_matches = 0
    if program_scores: max_matches = max(s['matches'] for s in program_scores.values())

    survivors = []
    if max_matches > 0:
        top_performers = sorted(
            [(lbl, s['weighted_score']) for lbl, s in program_scores.items() if s['matches'] == max_matches],
            key=lambda x: x[1], reverse=True
        )
        survivors = [lbl for lbl, _ in top_performers[:10]]

    if not survivors and program_scores:
        fallbacks = sorted(program_scores.items(), key=lambda kv: kv[1]['weighted_score'], reverse=True)
        survivors = [l for l, _ in fallbacks[:3]]

    return survivors

# --- ULTIMATE RANKING: Op-Type Diversity Enforcement (handles 3 solver types) ---
def rank_candidates_for_test_top_N(cand_pool, survivors, sig=None, N=2):
    scored_candidates = []

    for lbl, y in cand_pool:
        score = 0.0
        if lbl not in survivors: score += 500.0
        else: score -= 50.0

        if sig and sig["colors"] is not None:
            f = grid_features(y)
            score += abs(f["colors"] - sig["colors"]) * 1.5
            score += abs(f["ccs"]    - sig["ccs"])    * 1.0
            if sig["filled"] > 0: score += abs(f["filled"] - sig["filled"]) / max(1, sig["filled"]) * 2.0

        scored_candidates.append({'score': score, 'pred_arr': y, 'lbl': lbl})

    scored_candidates.sort(key=lambda x: x['score'])

    unique_preds = []
    seen_keys = set()
    seen_categories = set()

    if scored_candidates:
        pred_1 = scored_candidates[0]
        pred_arr_1 = pred_1['pred_arr']
        unique_preds.append(pred_arr_1)
        seen_keys.add(pred_arr_1.tobytes())
        seen_categories.add(get_op_category(pred_1['lbl']))

    if N == 2 and len(unique_preds) == 1:
        H, W = pred_arr_1.shape
        pred_arr_2 = None
        DIVERSITY_THRESHOLD = max(5, int(0.08 * H * W))

        for cand in scored_candidates:
            current_arr = cand['pred_arr']
            key = current_arr.tobytes()
            op_cat = get_op_category(cand['lbl'])

            if key in seen_keys: continue

            is_structurally_diverse = (current_arr.shape == pred_arr_1.shape and hamming(current_arr, pred_arr_1) > DIVERSITY_THRESHOLD)
            is_functionally_diverse = (op_cat not in seen_categories)

            if is_structurally_diverse and is_functionally_diverse:
                pred_arr_2 = current_arr
                unique_preds.append(pred_arr_2)
                break

        if pred_arr_2 is None:
            for cand in scored_candidates:
                current_arr = cand['pred_arr']
                key = current_arr.tobytes()
                if key not in seen_keys and current_arr.shape == pred_arr_1.shape and hamming(current_arr, pred_arr_1) > DIVERSITY_THRESHOLD:
                    pred_arr_2 = current_arr
                    unique_preds.append(pred_arr_2)
                    break

        if pred_arr_2 is None and sig and sig.get("best_grid"):
            pred_arr_2 = to_np(sig["best_grid"])
            if pred_arr_2.shape != (H, W): pred_arr_2 = pad_to(pred_arr_2, H, W)

            if hamming(pred_arr_2, pred_arr_1) < DIVERSITY_THRESHOLD:
                pred_arr_2 = op_rotate90(pred_arr_1, H, W)

            if pred_arr_2.tobytes() not in seen_keys:
                unique_preds.append(pred_arr_2)
            else:
                unique_preds.append(op_mirror_h(pred_arr_1, H, W))

    while len(unique_preds) < N:
        unique_preds.append(unique_preds[0])

    return unique_preds[:N]

# Cell 6: SOLVER + I/O (Time-Optimized Orchestration) — UPDATED PER-INPUT SHAPE + IDENTITY BACKSTOP
def solve_task_arc_prize(task, time_limit=MAX_TIME_PER_TASK):
    t0=time.time()
    train_pairs=[(to_np(p["input"]), to_np(p["output"])) for p in task["train"]]
    test_inputs=[to_np(p["input"]) for p in task["test"]]

    # Keep global H,W for survivor selection only
    H_sel,W_sel = infer_target_shape(train_pairs)

    # Allocate time to survivor search for stability
    survivors = select_survivor_labels(train_pairs, H_sel, W_sel, USE_COMPOSITIONS, time_limit=max(1.5, time_limit*0.75))

    sig = learn_output_signature(train_pairs)

    predictions = []
    for x in test_inputs:
        # Infer the correct target shape for THIS test input
        Hx, Wx = infer_target_shape_for_input(train_pairs, x)

        time_left = time_limit - (time.time() - t0)
        # Minimum time check for test case execution
        if time_left < 0.05:
            default_pred = pad_to(x, Hx, Wx)
            identity_pred = pad_to(x, x.shape[0], x.shape[1])
            predictions.append([{"output": to_list(default_pred)}, {"output": to_list(identity_pred)}])
            continue

        # Generate candidates from ALL three solvers (A, B, C) with per-input target size
        cand_pool = generate_candidates(x, Hx, Wx, train_pairs, use_compositions=USE_COMPOSITIONS)
        filtered_cand_pool = filter_by_signature(cand_pool, sig, tol_colors=1, tol_ccs=1, tol_filled_ratio=0.6)

        # Rank the combined, filtered candidates with Op-Type Diversity
        top_2_preds = rank_candidates_for_test_top_N(filtered_cand_pool, survivors, sig=sig, N=2)

        # Identity backstop at exact input size
        identity_pred = pad_to(x, x.shape[0], x.shape[1])

        pred_1_arr = top_2_preds[0] if len(top_2_preds) >= 1 else pad_to(x, Hx, Wx)
        pred_2_arr = top_2_preds[1] if len(top_2_preds) >= 2 else identity_pred

        # If pred_2 equals pred_1, force identity fallback to guarantee diversity
        if pred_2_arr.shape == pred_1_arr.shape and np.array_equal(pred_2_arr, pred_1_arr):
            pred_2_arr = identity_pred

        predictions.append([
            {"output": to_list(pred_1_arr)},
            {"output": to_list(pred_2_arr)}
        ])

    return predictions

def solve_many(tasks):
    outs=[]
    for i,task_data in enumerate(tasks):
        if (i % PRINT_PROGRESS_EVERY)==0:
            print(f"Solving task {i+1}/{len(tasks)}")
        try:
            preds=solve_task_arc_prize(task_data, time_limit=MAX_TIME_PER_TASK)
        except Exception:
            # Fallback to a single default prediction in case of exception during solve
            preds=[[{"output": [[0]]}, {"output": [[0]]}]] * len(task_data.get("test", []))
        outs.append(preds)
    return outs

# ----------------------------------------------------------------
# Local Evaluation Function
# ----------------------------------------------------------------
def evaluate_on_training(tasks):
    """
    Evaluates local performance by scoring the number of correct OUTPUTS
    (puzzles solved) using BOTH attempts, mimicking the competition.
    """
    total_outputs = 0
    correct_outputs = 0

    print("\n--- Running New, Corrected Local Proxy Accuracy Check ---")

    for i, task in enumerate(tasks):
        fake_task={"train": task["train"], "test": [{"input": t["input"]} for t in task["train"]]}
        preds_list_of_lists = solve_task_arc_prize(fake_task, time_limit=MAX_TIME_PER_TASK * 1.5)
        y_true_list=[to_np(t["output"]) for t in task["train"]]

        for j, y_true in enumerate(y_true_list):
            total_outputs += 1
            if j >= len(preds_list_of_lists): continue
            pred_list = preds_list_of_lists[j]

            if len(pred_list) < 2 or "output" not in pred_list[0] or "output" not in pred_list[1]:
                 continue

            try:
                 p1 = to_np(pred_list[0]["output"])
                 p2 = to_np(pred_list[1]["output"])

                 is_p1_correct = (p1.shape == y_true.shape and hamming(p1, y_true) == 0)
                 is_p2_correct = (p2.shape == y_true.shape and hamming(p2, y_true) == 0)

                 if is_p1_correct or is_p2_correct:
                     correct_outputs += 1
            except Exception:
                 continue  # Ignore tasks that fail numpy conversion

    accuracy = correct_outputs / total_outputs if total_outputs > 0 else 0
    return accuracy, correct_outputs, total_outputs

# Cell 7: Data Loading & Local Evaluation
def load_keyed_arc_tasks(path):
    """Loads ARC tasks from path, ensuring they are keyed by their IDs."""
    with open(path, "r") as f:
        raw = f.read().strip()

    if not (raw.startswith("{") or raw.startswith("[")):
        tasks_list = load_arc_tasks_anyshape(path)
        return {f"task_{i:03d}": t for i, t in enumerate(tasks_list)}

    obj = json.loads(raw)

    if isinstance(obj, dict):
        return {k: v for k, v in obj.items() if 'train' in v or 'test' in v}

    tasks_list = load_arc_tasks_anyshape(path)
    return {f"task_{i:03d}": t for i, t in enumerate(tasks_list)}

# 1. Load training data as a list for the local evaluation
train_tasks = load_arc_tasks_anyshape(TRAIN_JSON)
print("Training tasks (normalized):", len(train_tasks))

# 2. CRITICAL: Load test data as a DICTIONARY to preserve the Task IDs
test_tasks_dict = load_keyed_arc_tasks(TEST_JSON)
print("Test tasks (normalized):", len(test_tasks_dict))
test_tasks_list = list(test_tasks_dict.values())
test_task_ids = list(test_tasks_dict.keys())

# Sanity check
if train_tasks:
    print(f"<class 'dict'> {train_tasks[0].keys()}")

# 3. Proxy evaluation
local_accuracy, solved_outputs, total_outputs = evaluate_on_training(train_tasks)
print("\n--- Local Accuracy Report (FINAL RECURSIVE MULTI-MODAL ENGINE) ---")
print(f"Total training outputs (puzzles) checked: {total_outputs}")
print(f"Correctly solved outputs (by either attempt): {solved_outputs}")
print(f"**Local Proxy Accuracy:** {local_accuracy:.4f} (Goal: 0.8500+)")  # Ignore this score!

# 4. Run the full solver on the test set
print("\n--- Running Full Test Solver ---")
test_predictions = solve_many(test_tasks_list)
print("Full test solver complete.")

# Cell 8: Submission Writer
def write_submission(task_ids, predictions, path):
    """
    Formats the predictions into the required Kaggle submission JSON structure.
    """
    submission_dict = {}

    for task_id, task_preds in zip(task_ids, predictions):

        # NOTE: The predictions format from solve_many is:
        # [[{'output': grid1}, {'output': grid2}], [{'output': grid1}, {'output': grid2}], ...]

        formatted_preds = []
        for pred_pair in task_preds:

            # Ensure the structure is right, default if not
            a1 = pred_pair[0].get("output", [[0]])
            a2 = pred_pair[1].get("output", [[0]])

            formatted_preds.append({
                "attempt_1": a1,
                "attempt_2": a2,
            })

        submission_dict[task_id] = formatted_preds

    with open(path, "w") as f:
        json.dump(submission_dict, f)

    # Final structure check
    if all(len(v)==len(test_tasks_dict[k]["test"]) for k,v in submission_dict.items()):
          print("Submission structure OK ✅ — Go ahead and Submit to Competition.")
    else:
          print("WARNING: Submission structure mismatch detected.")

# Write the final submission file
write_submission(test_task_ids, test_predictions, SUBMISSION_PATH)

print(f"Wrote submission to: {SUBMISSION_PATH}")

# Cell 9: Final Submission Sanity Check
print("--- Running FINAL Submission File Sanity Check ---")

try:
    with open(SUBMISSION_PATH, 'r') as f:
        final_submission = json.load(f)

    task_ids = list(final_submission.keys())

    # 1. Check total tasks
    print(f"\n✅ Total tasks found in submission: {len(task_ids)}")
    print(f"   (Expected: 240, matching the test set)")

    # 2. Check a random sample task structure
    if task_ids:
        sample_task_id = task_ids[0]
        sample_preds = final_submission[sample_task_id]

        print(f"\n✅ Sample Task ID Checked: {sample_task_id}")

        # Check overall prediction count
        expected_test_cases = len(test_tasks_dict[sample_task_id]["test"])
        print(f"   - Total test cases predicted: {len(sample_preds)}")
        print(f"   - Expected test cases (from data): {expected_test_cases}")

        # Check the structure of the FIRST prediction in the list
        if sample_preds and isinstance(sample_preds, list):
            first_pred = sample_preds[0]

            # Check for required keys (attempt_1, attempt_2)
            required_keys = ["attempt_1", "attempt_2"]
            if all(k in first_pred for k in required_keys):
                print(f"   - Prediction keys are correct: {list(first_pred.keys())}")

                # Check the content type (should be a list of lists)
                a1 = first_pred['attempt_1']
                a2 = first_pred['attempt_2']

                is_a1_grid = isinstance(a1, list) and (not a1 or isinstance(a1[0], list))
                is_a2_grid = isinstance(a2, list) and (not a2 or isinstance(a2[0], list))

                if is_a1_grid and is_a2_grid:
                    print("   - Attempt content is correctly formatted as List[List[int]]")
                    print("\n--- Sample Prediction Content (First Test Case) ---")
                    print(f"Task ID: {sample_task_id}")
                    # Safety check for empty lists
                    a1_h = len(a1); a1_w = len(a1[0]) if a1 and a1[0] else 0
                    a2_h = len(a2); a2_w = len(a2[0]) if a2 and a2[0] else 0
                    print(f"Attempt 1 Grid Shape: {a1_h}x{a1_w}")
                    print(f"Attempt 2 Grid Shape: {a2_h}x{a2_w}")
                    # Print only the first 2 rows for brevity
                    print(f"Attempt 1 (Snippet): {a1[:2]}")
                    print(f"Attempt 2 (Snippet): {a2[:2]}")
                    print("-----------------------------------------------------")
                else:
                    print("❌ ERROR: Attempt content is NOT a list of lists (the grid format).")

            else:
                print(f"❌ ERROR: Missing required keys 'attempt_1' or 'attempt_2'. Keys found: {list(first_pred.keys())}")
        else:
            print("❌ ERROR: Predictions for the task are not in a List format.")

except FileNotFoundError:
    print(f"❌ ERROR: Submission file not found at {SUBMISSION_PATH}. Did Cell 8 run successfully?")
except json.JSONDecodeError:
    print("❌ ERROR: Submission file is not valid JSON.")
except Exception as e:
    print(f"❌ An unexpected error occurred during check: {e}")

Training tasks (normalized): 1000
Test tasks (normalized): 240
<class 'dict'> dict_keys(['train', 'test'])

--- Running New, Corrected Local Proxy Accuracy Check ---

--- Local Accuracy Report (FINAL RECURSIVE MULTI-MODAL ENGINE) ---
Total training outputs (puzzles) checked: 3232
Correctly solved outputs (by either attempt): 155
**Local Proxy Accuracy:** 0.0480 (Goal: 0.8500+)

--- Running Full Test Solver ---
Solving task 1/240
Solving task 51/240
Solving task 101/240
Solving task 151/240
Solving task 201/240
Full test solver complete.
Submission structure OK ✅ — Go ahead and Submit to Competition.
Wrote submission to: /kaggle/working/submission.json
--- Running FINAL Submission File Sanity Check ---

✅ Total tasks found in submission: 240
   (Expected: 240, matching the test set)

✅ Sample Task ID Checked: 00576224
   - Total test cases predicted: 1
   - Expected test cases (from data): 1
   - Prediction keys are correct: ['attempt_1', 'attempt_2']
   - Attempt content is correctly f